In [103]:
import getpass
import pandas as pd
import datetime
from pandas import ExcelWriter
from openpyxl import load_workbook



#Get SQL Report
username = getpass.getuser()
filename = "C:\\Users\\{0}\\Desktop\\RunIt\\LindenDollar.csv".format(username)
df = pd.read_csv(filename, parse_dates = ['TRANS_TS','COL13','COL20'])


#Transforming Datetime
df['Year'] = df.TRANS_TS.dt.year
df['Month'] = df.TRANS_TS.dt.month

In [104]:
#Sell Linden Dollar Table
df_BuySell = df.loc[:,'COL11':'COL19']
df_BuySell = df_BuySell[~df_BuySell['COL11'].isnull()]
df_BuySell['Year_'] = df_BuySell.COL13.dt.year
df_BuySell['Month_'] = df_BuySell.COL13.dt.month
df_BuySell.sort_values(by=['Year_', 'Month_'], ascending = False, inplace= True)


In [105]:
#Payout Linden Dollar Table
df_Payout = df.loc[:,'COL20':'COL23']
df_Payout =df_Payout[~df_Payout['COL20'].isnull()]
df_Payout['Year_Payout'] = df_Payout.COL20.dt.year
df_Payout['Month_Payout'] = df_Payout.COL20.dt.month
df_Payout.sort_values(by=['Year_Payout', 'Month_Payout'], ascending = False, inplace=True)


In [106]:
#Create Incoming and Outgoing Linden Dollars table by USD
table = df.pivot_table(index = ['Year', 'Month'], columns = 'TRANS_TYPE', aggfunc='sum', values = 'USD_CONVERT')
IncomingByMonth = pd.DataFrame(table['INCOMING'])
OutgoingByMonth = pd.DataFrame(table['OUTGOING'])
IncomingByMonth

INCOMING
Year   Month          
2020.0 1.0      930.08
       2.0     1477.41
       3.0     1196.18
       4.0     2456.69
       5.0     1911.38
       6.0     1670.27
       7.0     2059.15
       8.0     2745.68
       9.0     1393.99
       10.0    1359.12
       11.0    1587.05
       12.0    1364.50
2021.0 1.0     1876.95
       2.0      212.35

In [107]:
#Create Outgoing Linden Dollars table by USD
OutgoingByMonth

OUTGOING
Year   Month          
2020.0 1.0      906.87
       2.0     1081.96
       3.0     1624.61
       4.0     2311.83
       5.0     1908.12
       6.0     1854.17
       7.0     2011.02
       8.0     2637.74
       9.0     1194.52
       10.0    1471.85
       11.0    1656.59
       12.0    1375.68
2021.0 1.0     1305.91
       2.0      245.10

In [108]:
#Create Pivot Table for Incoming and Outgoing for CounterParty
Incoming_Table = df[df['TRANS_TYPE'] == 'INCOMING']
Outgoing_Table = df[df['TRANS_TYPE'] == 'OUTGOING']

Incoming_Pivot = Incoming_Table.pivot_table(index = ['COUNTERPARTY'], 
                                            columns = ['Year', 'Month'], 
                                            aggfunc='sum', 
                                            values = 'USD_CONVERT', 
                                            fill_value = 0, 
                                            margins = True).sort_values('All', ascending= False)
IncomingPivot = Incoming_Pivot[0:10] 

In [109]:
Outgoing_Pivot = Outgoing_Table.pivot_table(index =['COUNTERPARTY'], 
                                            columns = ['Year', 'Month'], 
                                            aggfunc = 'sum', 
                                            values = 'USD_CONVERT', 
                                            fill_value = 0, 
                                            margins = True).sort_values('All', ascending = False)

OutgoingPivot = Outgoing_Pivot[:10]

In [110]:
#Create Incoming Linden Dollars table by Type in USD

IncomingByType = Incoming_Table.pivot_table(index = ['TRANS_TYPE_NAME'],
                                            columns = ['Year', 'Month'],
                                            aggfunc = 'sum',
                                            values = 'USD_CONVERT',
                                            fill_value = 0,
                                            margins = True).sort_values('All', ascending = False)
IncomingByType = IncomingByType[:10]

In [111]:
#Create Outgoing Linden Dollars table by Type in USD
OutgoingByType = Outgoing_Table.pivot_table(index =['TRANS_TYPE_NAME'],
                                            columns = ['Year', 'Month'],
                                            fill_value = 0,
                                            aggfunc = 'sum',
                                            values = 'USD_CONVERT',
                                            margins = True).sort_values('All', ascending = False)
OutgoingByType = OutgoingByType[:10]

In [112]:
#Create NET Linden Dollars by Type in USD

NetByType = df.pivot_table(index = ['TRANS_TYPE_NAME'],
                           columns = ['Year', 'Month'],
                           values = 'NETINUSD',
                           fill_value = 0,
                           aggfunc = 'sum',
                           margins = True).sort_values('All', ascending = False)

In [113]:
#Net Linden Dollars in USD by Person
NetByPerson = df.pivot_table(index = ['COUNTERPARTY'],
                             columns = ['Year', 'Month'],
                             aggfunc = 'sum',
                             values = 'NETINUSD',
                             fill_value = 0,
                             margins = True).sort_values('All', ascending = False)

NetByPerson = NetByPerson.head(10).append(NetByPerson.tail(10))

In [114]:
#Buy Sell Linden Dollar Pivot Table
LdExchange = df_BuySell.pivot_table(index = ['COL15'], 
                                 columns = ['Year_', 'Month_'],
                                 aggfunc = 'sum',
                                 values = 'COL18').T


In [115]:
# Payout Table Pivot
PayoutPivot = df_Payout.pivot_table(index = 'COL21', 
                                  columns = ['Year_Payout', 'Month_Payout', 'COL22'],
                                  values = 'COL23',
                                  aggfunc = 'sum').T

In [116]:
#Load Workbook
filename = pd.ExcelWriter("C:\\Users\\{0}\\Desktop\\RunIt\\Analysis_.xlsx".format(username))
excelBook = load_workbook(filename)

#Read workbook and sheets
with pd.ExcelWriter(filename, engine='openpyxl') as writer:
    writer.book = excelBook
    writer.sheets = dict((ws.title, ws) for ws in excelBook.worksheets)
#Write to Excel tab
    IncomingByMonth.to_excel(writer, sheet_name='LDTransaction', startcol = 0, startrow = 4)
    OutgoingByMonth.to_excel(writer, sheet_name='LDTransaction', startcol = 5, startrow = 4)
    IncomingPivot.to_excel(writer, sheet_name='LDTransaction', startcol = 0, startrow=24)
    IncomingByType.to_excel(writer, sheet_name = 'LDTransaction', startcol = 0, startrow = 41)
    OutgoingPivot.to_excel(writer, sheet_name='LDTransaction', startcol = 0, startrow=58)
    OutgoingByType.to_excel(writer, sheet_name = 'LDTransaction', startcol=0, startrow= 75)
    NetByPerson.to_excel(writer, sheet_name = 'LDTransaction', startcol=0, startrow=92)
    NetByType.to_excel(writer, sheet_name='LDTransaction', startcol=0, startrow=118)
    LdExchange.to_excel(writer, sheet_name = 'BuySell', startcol=6, startrow=10)
    PayoutPivot.to_excel(writer, sheet_name= 'BuySell', startcol=6, startrow=26)
    writer.save()
    writer.close()